In [1]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import datetime
import boto3

In [3]:
df = pd.read_csv("Walmart_customer_purchases.csv")

In [4]:
df.head()

,Customer_ID,Age,Gender,City,Category,Product_Name,Purchase_Date,Purchase_Amount,Payment_Method,Discount_Applied,Rating,Repeat_Customer
0,84607c1f-910c-44d5-b89f-e1ee06dd34c0,49,Female,New Cynthia,Electronics,Smartphone,2024-08-30,253.26,Cash on Delivery,No,1,Yes
1,f2a81712-a73e-4424-8b39-4c615a0bd4ea,36,Other,Cruzport,Clothing,T-Shirt,2024-12-21,73.19,Debit Card,Yes,1,No
2,da9be287-8b0e-4688-bccd-1a2cdd7567c6,52,Male,Jeffreytown,Beauty,Perfume,2024-12-26,125.62,Credit Card,Yes,1,No
3,50ec6932-3ac7-492f-9e55-4b148212f302,47,Female,Jenniferburgh,Electronics,Smartwatch,2024-11-04,450.32,Credit Card,No,2,Yes
4,8fdc3098-fc75-4b0f-983c-d8d8168c6362,43,Other,Kingshire,Electronics,Smartphone,2024-10-07,369.28,Credit Card,Yes,2,Yes


In [5]:
df.describe()

,Age,Purchase_Amount,Rating
count,50000.000000,50000.000000,50000.000000
mean,38.945220,255.532230,2.998680
std,12.398137,141.574416,1.417956
min,18.000000,10.010000,1.000000
25%,28.000000,133.050000,2.000000
50%,39.000000,255.045000,3.000000
75%,50.000000,378.912500,4.000000
max,60.000000,499.990000,5.000000


As we can see, The Average age of customer is around 39 years. Average Order Value is $255 and Avergae Product Rating is 3.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Customer_ID       50000 non-null  object 
 1   Age               50000 non-null  int64  
 2   Gender            50000 non-null  object 
 3   City              50000 non-null  object 
 4   Category          50000 non-null  object 
 5   Product_Name      50000 non-null  object 
 6   Purchase_Date     50000 non-null  object 
 7   Purchase_Amount   50000 non-null  float64
 8   Payment_Method    50000 non-null  object 
 9   Discount_Applied  50000 non-null  object 
 10  Rating            50000 non-null  int64  
 11  Repeat_Customer   50000 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 4.6+ MB


We dont have any missing data.Thats good

### Data Cleaning & Feature Engineering Summary

- **Converted `Purchase_Date` to datetime**  
  → Enables time-based analysis and revenue trends.

- **Created `Month` column**  
  → Helps group revenue and purchases by month.

- **Created `Age_Group` buckets**  
  → Useful for customer segmentation by demographics.

- **Converted `Purchase_Amount` to numeric**  
  → Ensures accurate calculations for KPIs like revenue and AOV.

- **Normalized text fields (`City`, `Category`, `Product_Name`, `Gender`)**  
  → Prevents duplication and ensures consistent grouping in analysis.

- **Exported cleaned data as `walmart_sales_cleaned.csv`**  
  → Ready for upload to AWS S3 and loading into Redshift.



In [7]:
df['Purchase_Date'] = pd.to_datetime(df['Purchase_Date'])

In [8]:
df['Month'] = df['Purchase_Date'].dt.to_period('M')
df['Age_Group'] = pd.cut(
    df['Age'],
    bins=[0, 18, 25, 35, 45, 55, 65, 100],
    labels=['<18', '18-25', '26-35', '36-45', '46-55', '56-65', '65+'],
    right=False
)

In [9]:
for col in ['City', 'Category', 'Product_Name', 'Gender']:
    df[col] = df[col].str.title().str.strip()

df['Discount_Applied'] = df['Discount_Applied'].str.capitalize().str.strip()
df['Repeat_Customer'] = df['Repeat_Customer'].str.capitalize().str.strip()


In [10]:
df.head()

,Customer_ID,Age,Gender,City,Category,Product_Name,Purchase_Date,Purchase_Amount,Payment_Method,Discount_Applied,Rating,Repeat_Customer,Month,Age_Group
0,84607c1f-910c-44d5-b89f-e1ee06dd34c0,49,Female,New Cynthia,Electronics,Smartphone,2024-08-30,253.26,Cash on Delivery,No,1,Yes,2024-08,46-55
1,f2a81712-a73e-4424-8b39-4c615a0bd4ea,36,Other,Cruzport,Clothing,T-Shirt,2024-12-21,73.19,Debit Card,Yes,1,No,2024-12,36-45
2,da9be287-8b0e-4688-bccd-1a2cdd7567c6,52,Male,Jeffreytown,Beauty,Perfume,2024-12-26,125.62,Credit Card,Yes,1,No,2024-12,46-55
3,50ec6932-3ac7-492f-9e55-4b148212f302,47,Female,Jenniferburgh,Electronics,Smartwatch,2024-11-04,450.32,Credit Card,No,2,Yes,2024-11,46-55
4,8fdc3098-fc75-4b0f-983c-d8d8168c6362,43,Other,Kingshire,Electronics,Smartphone,2024-10-07,369.28,Credit Card,Yes,2,Yes,2024-10,36-45


In [11]:
# Generate dynamic file name
today = datetime.datetime.today()
filename = f"walmart_customer_{today.strftime('%b-%Y')}.csv"
df.to_csv(filename, index = False, encoding='utf-8')

In [14]:
log_data = {
    "last_run": today.strftime('%Y-%m-%d %H:%M:%S'),
    "last_file": filename
}

In [15]:
# Upload to S3
bucket_name = 'walmart-sales-dashboard' 
s3_key = f"monthly_data/{filename}" 
# Create S3 client
s3 = boto3.client('s3')

# Upload file
try:
    s3.upload_file(filename, bucket_name, s3_key)
    print(f"File uploaded successfully to s3://{bucket_name}/{s3_key}")
except Exception as e:
    print(f"Upload failed: {e}")

File uploaded successfully to s3://walmart-sales-dashboard/monthly_data/walmart_customer_Apr-2025.csv


## Exporting the views from Amazon Redshift

In [17]:
pip install psycopg2-binary pandas


  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/61/69/3b3d7bd583c6d3cbe5100802efa5beacaacc86e37b653fc708bf3d6853b8/psycopg2_binary-2.9.10-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ------ --------------------------------- 0.2/1.2 MB 3.9 MB/s eta 0:00:01
   --------------------------------- ------ 1.0/1.2 MB 10.2 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 9.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [29]:
import psycopg2
import pandas as pd

# Replace with your Redshift credentials
config = {
    'host': 'walmart-bie-workgroup.748065983427.us-east-1.redshift-serverless.amazonaws.com',
    'port': '5439',
    'user': 'admin',
    'password': 'Walmart_bie_2025',
    'database': 'dev'
}

views = [
    'vw_category_revenue',
    'vw_monthly_revenue_by_category',
    'vw_repeat_customer_comparison',
    'vw_discount_behavior',
    'vw_city_revenue',
    'vw_payment_method_summary'
]

try:
    conn = psycopg2.connect(**config)
    for view in views:
        df = pd.read_sql_query(f"SELECT * FROM {view}", conn)
        df.to_csv(f"{view}.csv", index=False)
        print(f"✅ Exported {view}.csv")
    conn.close()
except Exception as e:
    print(f"❌ Error: {e}")


C:\Users\chand\AppData\Local\Temp\ipykernel_81496\3518519641.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {view}", conn)


✅ Exported vw_category_revenue.csv


C:\Users\chand\AppData\Local\Temp\ipykernel_81496\3518519641.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {view}", conn)


✅ Exported vw_monthly_revenue_by_category.csv


C:\Users\chand\AppData\Local\Temp\ipykernel_81496\3518519641.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {view}", conn)


...

✅ Exported vw_discount_behavior.csv


C:\Users\chand\AppData\Local\Temp\ipykernel_81496\3518519641.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {view}", conn)


✅ Exported vw_city_revenue.csv


C:\Users\chand\AppData\Local\Temp\ipykernel_81496\3518519641.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {view}", conn)


✅ Exported vw_payment_method_summary.csv
